Import libraries

In [1]:
import torch
import cv2
%reload_ext autoreload
%autoreload 2
# this is the main library used (sits on top of PyTorch)
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from sklearn import metrics

Set up data: PATH should lead to the data folder with subfolders train, valid, and test. Train and valid should each have subfolders "yes" and "no". For this to run, you need something in these "yes" and "no" folders, though it won't affect the predictions--just put in some images from your test data.

In [2]:
# where the data is
PATH = "data\e_UMHS-0018-0002.day-04_BD1-BD2_transpose"
# using resnet architecture
arch = resnet34
# size of square image in pixels
sz = 44
# transforms used on training data
transforms_up_down = [RandomScale(sz,1.2),RandomRotate(1)]
tfms = tfms_from_model(arch,sz,crop_type = CropType.NO,aug_tfms=transforms_up_down)
# data: comes from PATH, used tfms on training data, bs of 8 for training data, test data located in test folder
data = ImageClassifierData.from_paths(PATH,tfms=tfms,bs=8,test_name='test')
# load in pretraine
state = torch.load('saved_model.pkl',map_location=torch.device('cpu')) # remove map_location parameter if on GPU
learn2 = ConvLearner.pretrained(arch,data,precompute=False)
learn2.model.load_state_dict(state)

<All keys matched successfully>

In [3]:
log_preds_test = learn2.predict(is_test=True)
preds_test = np.argmax(log_preds_test,axis=1)
probs_test = np.exp(log_preds_test[:,1])

In [4]:
test_names = np.empty_like(data.test_ds.fnames)
for i in range(len(data.test_ds.fnames)):
    test_names[i] = data.test_ds.fnames[i]
    #temp = data.test_ds.fnames[i]
    #matchobj = re.search('.*im.*',temp)
    #test_names[i] = matchobj.group()
test_df = pd.DataFrame(data = test_names,columns = ['image_number'])
test_df['prediction'] = preds_test
test_df['probability'] = probs_test

In [5]:
print(test_df.to_string())

           image_number  prediction   probability
0        test\img_1.jpg           0  4.820401e-07
1       test\img_10.jpg           0  2.791198e-04
2      test\img_100.jpg           0  1.395540e-03
3     test\img_1000.jpg           0  1.740564e-03
4     test\img_1001.jpg           0  3.656535e-04
5     test\img_1002.jpg           0  2.988446e-05
6     test\img_1003.jpg           0  1.206613e-05
7     test\img_1004.jpg           0  2.436792e-05
8     test\img_1005.jpg           0  7.931794e-05
9     test\img_1006.jpg           0  1.177155e-03
10    test\img_1007.jpg           0  1.493382e-02
11    test\img_1008.jpg           0  6.609930e-04
12    test\img_1009.jpg           0  6.057288e-04
13     test\img_101.jpg           0  1.019288e-01
14    test\img_1010.jpg           0  5.406250e-05
15    test\img_1011.jpg           0  1.709736e-03
16    test\img_1012.jpg           0  1.175994e-04
17    test\img_1013.jpg           0  1.045228e-04
18    test\img_1014.jpg           0  4.867750e-06
